In [1]:
import datetime
import openpyxl
from datetime import timedelta, date, time
import calendar
import pandas as pd
import os
import numpy as np
import fnmatch
from dateutil.relativedelta import relativedelta
from math import sqrt, exp, log, pi
import math
from scipy.stats import norm

import warnings
warnings.filterwarnings("ignore")


def d(sigma, S, K, r, q, t):
    d1 = 1 / (sigma * sqrt(t)) * ( log(S/K) + (r - q + sigma**2/2) * t)
    d2 = d1 - sigma * sqrt(t)
    return d1, d2

def call_price(sigma, S, K, r, q, t, d1, d2):
    C = norm.cdf(d1) * S * exp(-q * t)- norm.cdf(d2) * K * exp(-r * t)
    return C

def put_price(sigma, S, K, r, q, t, d1, d2):
    P = - S * exp(-q * t) * norm.cdf(-d1) + K * exp(-r * t) * norm.cdf(-d2)
    return P

def call_IV(S, K, r, q, t, C0):
    #  Tolerances  
    tol = 1e-3
    epsilon = 1

    #  Variables to log and manage number of iterations
    count = 0
    max_iter = 1000


    #  We need to provide an initial guess for the root of our function
    vol = 0.50

    while epsilon > tol:
        #  Count how many iterations and make sure while loop doesn't run away
        count += 1
        if count >= max_iter:
            print('Breaking on count',vol,vega,epsilon,function_value)
            break;

        #  Log the value previously calculated to computer percent change between iterations
        orig_vol = vol

        #  Calculate the vale of the call price
        d1, d2 = d(vol, S, K, r,q, t)
        function_value = call_price(vol, S, K, r, q, t, d1, d2) - C0

        #  Calculate vega, the derivative of the price with respect to volatility
        vega = S * norm.pdf(d1) * sqrt(t)* exp(-q * t)

        #  Update for value of the volatility
        vol = -function_value / vega + vol

        #  Check the percent change between current and last iteration
        epsilon = abs( (vol - orig_vol) / orig_vol )
        
    return vol
    #  Print out the results
#     print('Sigma = ', vol)
#     print('Code took ', count, ' iterations')


# This Python code can be used to calculate the value for Implied Volatility for a European put

def put_IV(S, K, r, q, t, P0):
    #  Tolerances
    tol = 1e-3
    epsilon = 1

    #  Variables to log and manage number of iterations
    count = 0
    max_iter = 1000

    #  We need to provide an initial guess for the root of our function

    vol = 0.50
    while epsilon > tol:
        #  Count how many iterations and make sure while loop doesn't run away
        count += 1
        if count >= max_iter:
            print('Breaking on count',vol,vega,epsilon,function_value)
            break;

        #  Log the value previously calculated to computer percent change
        #  between iterations
        orig_vol = vol

        #  Calculate the vale of the call price
        d1, d2 = d(vol, S, K, r,q, t)
        function_value = put_price(vol, S, K, r, q, t, d1, d2) - P0

        #  Calculate vega, the derivative of the price with respect to
        #  volatility
        vega = S * norm.pdf(d1) * sqrt(t)* exp(-q * t)

        #  Update for value of the volatility
        vol = -function_value / vega + vol

        #  Check the percent change between current and last iteration
        epsilon = abs( (vol - orig_vol) / orig_vol )

    return vol

def DeltaC (S, K, r, q,t, call_vol):
    d1, d2 = d(call_vol, S, K, r,q, t)
    DC = exp(-q * t) * norm.cdf(d1)
    return DC

def DeltaP (S, K,r,q, t, put_vol):
    d1, d2 = d(put_vol, S, K, r,q, t)
    DP = -exp(-q * t) * norm.cdf(-d1)
    return DP

# Inputs

main_folder = "D:\\Options\\GDFL Raw Data"
instrument = "NIFTY"
TodayDate = ''
Year = "2021"
Month = "JUN_2021"
DateList = ['11062021']

#DateList = ['14072022','15072022','18072022','19072022','20072022','21072022','22072022','25072022','26072022','27072022','28072022','29072022']

#DateList = ['24022022', '25022022', '28022022']
#DateList = [ '02032022', '03032022','31032022']
#DateList = [ '04032022', '07032022', '08032022', '09032022', '10032022', '11032022', '14032022', '15032022', '16032022', '17032022', '21032022', '22032022', '23032022', '24032022', '25032022', '28032022', '29032022','30032022','31032022']

#DateList = [ '02032022', '03032022', '04032022', '07032022', '08032022', '09032022', '10032022', '11032022', '14032022', '15032022', '16032022', '17032022', '21032022', '22032022', '23032022', '24032022', '25032022', '28032022', '29032022','30032022']

#DateList = ['01022022', '02022022', '03022022', '04022022', '07022022', '08022022', '09022022', '10022022', '11022022', '14022022', '15022022', '16022022', '17022022', '18022022', '21022022', '22022022', '23022022', '24022022', '25022022', '28022022']
#DateList = ['01022021', '02022021', '03022021', '04022021', '05022021', '08022021', '09022021', '10022021', '11022021', '12022021', '15022021', '16022021', '17022021', '18022021', '19022021', '22022021', '23022021', '24022021', '25022021', '26022021']
#DateList = ['01032021', '02032021', '03032021', '04032021', '05032021', '08032021', '09032021', '10032021', '12032021', '15032021', '16032021', '17032021', '18032021', '19032021', '22032021', '23032021', '24032021', '25032021', '26032021', '30032021', '31032021']
#DateList = ['01042021', '05042021', '06042021', '07042021', '08042021', '09042021', '12042021', '13042021', '15042021', '16042021', '19042021', '20042021', '22042021', '23042021', '26042021', '27042021', '28042021', '29042021', '30042021']
#DateList = ['03052021', '04052021', '05052021', '06052021', '07052021', '10052021', '11052021', '12052021', '14052021', '17052021', '18052021', '19052021', '20052021', '21052021', '24052021', '25052021', '26052021', '27052021', '28052021', '31052021']
#DateList = ['01062021','02062021','03062021', '04062021','07062021','08062021','09062021','10062021','11062021','14062021','15062021','16062021','17062021','18062021','21062021','22062021','23062021','24062021','25062021','28062021','29062021','30062021']
#DateList = ['01072021','02072021','05072021','06072021','07072021','08072021','09072021','12072021','13072021','14072021','15072021','16072021','19072021','20072021','22072021','23072021','26072021','27072021','28072021','29072021','30072021']
#DateList = ['02082021','03082021','04082021','05082021','06082021','09082021','10082021','11082021','12082021','13082021','16082021','17082021','18082021','20082021','23082021','24082021','25082021','26082021','27082021','30082021']
#DateList = ['01092021','02092021','03092021','06092021','07092021','08092021','09092021','13092021','14092021','15092021','16092021','17092021','20092021','20092021','21092021','22092021','23092021','24092021','27092021','28092021','29092021','30092021']
#DateList = ['01102021','04102021','05102021','06102021','07102021','08102021','11102021','12102021','13102021','14102021','18102021','19102021','20102021','21102021','22102021','25102021','26102021','27102021','28102021','29102021']
#DateList = ['01112021','02112021','03112021','08112021','09112021','10112021','11112021','12112021','15112021','16112021','17112021','18112021','22112021','23112021','24112021','25112021','26112021','29112021','30112021']
#DateList = ['01122021','02122021','03122021','06122021','07122021','08122021','09122021','10122021','13122021','14122021','15122021','16122021','17122021','20122021','21122021','22122021','23122021','24122021','27122021','28122021','29122021','30122021','31122021']
#DateList = ['03012022','04012022','05012022','06012022','07012022','10012022','11012022','12012022','13012022','14012022','17012022','18012022','19012022','20012022','21012022','24012022','25012022','27012022','28012022','31012022']

for dates in DateList: 
    TodayDate = dates
    future_filename =str(instrument) + "-I.NFO.csv"
    future_filename = main_folder + "\\" + str(Year) + "\\" + str(Month) + "\\" + "GFDLNFO_TICK_" + str(TodayDate)  + "\\" + "GFDLNFO_TICK_" + str(TodayDate) +"\\"+ future_filename
    time_template_filename = 'TimeTemplate.csv'
    time_template_filename = main_folder + "\\" + time_template_filename 

    RiskFreeRate = 0.036
    Strike_increase = 50
    Strike_step = 50

    fut_data = pd.read_csv(future_filename)
    time_template = pd.read_csv(time_template_filename)

    # Calculating nearest expiry

    Holiday = ["02-04-2020", "11-03-2021", "13-05-2021", "19-08-2021", "04-11-2021", "26-01-2022", "01-03-2022", "18-03-2022", "14-04-2022", "15-04-2022", "03-05-2022", "09-08-2022", "15-08-2022", "31-08-2022", "05-10-2022", "24-10-2022", "26-10-2022", "08-11-2022"]
    for i in range(len(Holiday)):
        Holiday[i] = datetime.date(int(Holiday[i][6:10]), int(Holiday[i][3:5]), int(Holiday[i][0:2]))

    Data_date = fut_data.iloc[1].Date
    Data_date = datetime.date(int(Data_date[6:10]), int(Data_date[3:5]), int(Data_date[0:2]))
    CurrentDate = Data_date.strftime("%d%m%Y")

    next_thursday = Data_date + timedelta((calendar.THURSDAY-Data_date.weekday()) % 7 )
    if next_thursday in Holiday:
        next_thursday = next_thursday - timedelta(days = 1)
    NearestExpiry = next_thursday.strftime("%d%b%y")

    # Calculating days to expiry

    daystoexpiry = (datetime.datetime.strptime(NearestExpiry, "%d%b%y").date() - 
            datetime.datetime.strptime(CurrentDate, "%d%m%Y").date()) + timedelta(days=1)

    #Finding the range of the Index (future) for the day

#    min_strike=int(math.floor(fut_data['LTP'].min()/Strike_step))*Strike_step -100
#    max_strike=int(math.ceil(fut_data['LTP'].max()/Strike_step))*Strike_step +100   
  

    min_strike=15650
    max_strike=15700   
    
    CurrentStrike = min_strike


    fut_data_unique = fut_data.drop_duplicates(['Time'])
    fut_data_unique = fut_data_unique.add_suffix('_fut')
    fut_data_unique = fut_data_unique.rename(index=str, columns={'Time_fut':'Time'})
    fut_data_unique = fut_data_unique[['Time','BuyPrice_fut','SellPrice_fut']]
    final_data = time_template.merge(fut_data_unique,left_on = ['Time'], right_on = ['Time'], how = 'left')
    final_data = final_data.fillna(method='ffill')

    result = time_template
    

    for i in range(CurrentStrike, max_strike+Strike_increase, Strike_increase):
        print(TodayDate + str(i))
        CE_data = pd.read_csv(main_folder + "\\" + str(Year) + "\\" + str(Month) + "\\" + "GFDLNFO_TICK_" + str(TodayDate) + "\\" + "GFDLNFO_TICK_" + str(TodayDate) + "\\" +  instrument + NearestExpiry.upper()+ str(i) + 'CE.NFO.csv')
        PE_data = pd.read_csv(main_folder + "\\" + str(Year) + "\\" + str(Month) + "\\" + "GFDLNFO_TICK_" + str(TodayDate) + "\\" +  "GFDLNFO_TICK_" + str(TodayDate) + "\\" + instrument + NearestExpiry.upper()+ str(i) + 'PE.NFO.csv')

        
        CE_data_unique = CE_data.drop_duplicates(['Time'])
        PE_data_unique = PE_data.drop_duplicates(['Time'])

        CE_data_unique = CE_data_unique.add_suffix('_CE')
        CE_data_unique = CE_data_unique.rename(index=str, columns={'Time_CE':'Time'})

        PE_data_unique = PE_data_unique.add_suffix('_PE')
        PE_data_unique = PE_data_unique.rename(index=str, columns={'Time_PE':'Time'})

        interim_data = time_template.merge(CE_data_unique, left_on = ['Time'], right_on = ['Time'], how = 'left')
        interim_data = interim_data.merge(PE_data_unique, left_on = ['Time'], right_on = ['Time'], how = 'left')

        interim_data = interim_data.fillna(method='ffill')

        interim_data['row_num'] = np.arange(len(interim_data))
        interim_data['row_num_max'] = max(interim_data['row_num'])
        interim_data['decay_factor'] = (interim_data['row_num'] +1)/(interim_data['row_num_max'] +1)
        percentiles = interim_data['decay_factor'].quantile([0,0.99]).values
        interim_data['decay_factor'] = np.clip(interim_data['decay_factor'],percentiles[0],percentiles[1])        
        
        interim_data = interim_data.drop(['row_num','row_num_max'],axis = 1)

        interim_data['Weekly_future'] = interim_data['LTP_CE'] - interim_data['LTP_PE'] + i * \
                                      np.exp(-1 * RiskFreeRate * ((daystoexpiry.days - interim_data['decay_factor'])/365))

        interim_data['spot'] =  interim_data['Weekly_future'] /np.exp(RiskFreeRate * ((daystoexpiry.days - interim_data['decay_factor'])/365))
        interim_data.to_csv('check_file.csv')
        interim_data['call_iv'] = interim_data.apply(lambda x: call_IV(x['spot'],i,RiskFreeRate,0,(daystoexpiry.days - x['decay_factor'])/365,x['LTP_CE']), axis=1)
        interim_data['put_iv'] = interim_data.apply(lambda x: put_IV(x['spot'],i,RiskFreeRate,0,(daystoexpiry.days - x['decay_factor'])/365,x['LTP_PE']), axis=1)

        interim_data['delta_c'] = interim_data.apply(lambda x: DeltaC(x['spot'],i,RiskFreeRate,0,(daystoexpiry.days - x['decay_factor'])/365,x['call_iv']), axis=1)
        interim_data['delta_p'] = interim_data.apply(lambda x: DeltaP(x['spot'],i,RiskFreeRate,0,(daystoexpiry.days - x['decay_factor'])/365,x['put_iv']), axis=1)

        interim_data = interim_data[['Time','BuyPrice_CE','SellPrice_CE','BuyPrice_PE',
                                     'SellPrice_PE','Weekly_future','call_iv','delta_c','delta_p']]

        interim_data.rename(columns = {
                                     'BuyPrice_CE' : str(i) + 'C_B',
                                     'SellPrice_CE': str(i) + 'C_S',
                                     'BuyPrice_PE' : str(i) + 'P_B',
                                     'SellPrice_PE': str(i) + 'P_S',
                                     'Weekly_future': str(i) + 'WFUT',
                                     'call_iv': str(i) + '_IV',
                                     'delta_c': str(i) + 'C_Delta',
                                     'delta_p': str(i) + 'P_Delta'
                                     }, inplace = True)

        result  = result.merge(interim_data, left_on = ['Time'], right_on = ['Time'], how = 'left')
        print('results done for', TodayDate + str(i))

    result_final = final_data.merge(result,left_on = ['Time'], right_on = ['Time'], how = 'left')
    result_final.rename(columns = {'BuyPrice_fut' : 'FUT_B','SellPrice_fut': 'FUT_S'}, inplace = True)
    result_final.to_csv(TodayDate + '_results.csv')

    

1106202115650
results done for 1106202115650
1106202115700
results done for 1106202115700
